In [3]:
import segmentation_utils as su
import cv2
import matplotlib.pyplot as plt
import os

In [6]:
def count_shape(info_image):
    width = info_image[2] - info_image[0]
    height = info_image[3] - info_image[1]

    return (width, height)

In [14]:
def convert_bounding_boxes_part1(image_path,infor_part1, label_path, dir_save_label, name_file):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    with open(label_path, 'r') as f:
        labels = f.readlines()[:90]
    with open(f"{dir_save_label}/{name_file}", "w") as file:
        for label in labels:
            parts = label.strip().split()
            class_id = int(parts[0])

            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:])

            x_center *= width
            y_center *= height 
            bbox_width *= width 
            bbox_height *= height 

            x_min_part1 = infor_part1[0]
            y_min_part1 = infor_part1[1]

            part1_shape = count_shape(infor_part1)

            width_part1 = part1_shape[0]
            height_part1 = part1_shape[1]

            x_center_new = (x_center - x_min_part1)/width_part1
            y_center_new = (y_center - y_min_part1)/height_part1

            bbox_width_new = bbox_width / width_part1
            bbox_height_new = bbox_height / height_part1

            
            file.write(f"{class_id} {x_center_new} {y_center_new} {bbox_width_new} {bbox_height_new}\n")

# Tạo ra bộ dataset image-label cho part đầu

In [ ]:
dir_image = "C:/Users/huyho/OneDrive/Desktop/autograde/implement/Yolov8/data/images/train"  #thay bằng thử mục chứa ảnh gốcgốc
dir_label = "C:/Users/huyho/OneDrive/Desktop/autograde/implement/Yolov8/data/labels/train"  #thay bằng thư mục chứ label gốcgốc
dir_save_image = "D:/part1_images"
dir_save_label = "D:/part1_labels"
i = 0
for file in os.listdir(dir_image):
    i += 1
    if i == 2341:
        print(file)
        break
    image = cv2.imread(f"{dir_image}/{file}")
    centroids = su.findSquareContourCentroid(image)
    sorted_points = su.sort_points_grid(centroids, row_threshold = 15)

    first_small_part, infor_1 = su.segmentation_first_small_part_info(image, sorted_points) #sử dụng hàm trong segmentation_utils.py để extract part1
    cv2.imwrite(f"{dir_save_image}/{file}",first_small_part)
    
    convert_bounding_boxes_part1(image_path=f"{dir_image}/{file}", infor_part1=infor_1, label_path=f"{dir_label}/{file[:-3]}txt", dir_save_label=dir_save_label, name_file=f"{file[:-3]}txt")


# Tạo bộ dataset cho những part sau

In [ ]:
dir_image = "C:/Users/huyho/OneDrive/Desktop/autograde/implement/Yolov8/data/images/train"  #thay bằng thử mục chứa ảnh gốcgốc
dir_label = "C:/Users/huyho/OneDrive/Desktop/autograde/implement/Yolov8/data/labels/train"  #thay bằng thư mục chứ label gốcgốc
dir_save_image = "D:/part1_images"   #part thứ mấy thì đổi tên file dir
dir_save_label = "D:/part1_labels"
i = 0
for file in os.listdir(dir_image):
    i += 1
    if i == 2341:
        print(file)
        break
    image = cv2.imread(f"{dir_image}/{file}")
    centroids = su.findSquareContourCentroid(image)
    sorted_points = su.sort_points_grid(centroids, row_threshold = 15)

    # first_small_part, infor_1 = su.segmentation_first_small_part_info(image, sorted_points) #sử dụng hàm trong segmentation_utils.py để extract part1
    cropped_images, cropped_infor = su.segmentation_3_parts_info(image, sorted_points)
    cropped_image = cropped_images[0]       #index 0: phần 1,  index1: phần 2, index2: phần 3, index 3: phần mã đề thi, sbd
    cropped_info = cropped_infor[0]
    cv2.imwrite(f"{dir_save_image}/{file}",cropped_image)
    
    convert_bounding_boxes_part1(image_path=f"{dir_image}/{file}", infor_part1=cropped_info, label_path=f"{dir_label}/{file[:-3]}txt", dir_save_label=dir_save_label, name_file=f"{file[:-3]}txt")